In [ ]:
import torch
from pytorch_transformers import BertForMaskedLM
from pytorch_transformers import BertTokenizer

In [ ]:
text = "[CLS] | played baseball with my friends at school yesterday [SEP]"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")#←BEATベースアンケースドを使用
words = tokenizer.tokenize(text)
print(words)

tokenizer.tokenizeのところは注意rがついてるのと、ついてないの

ここから文章の一部をマスクする

In [ ]:
msk_idx = 3#3番目0からカウント
words[msk_idx] = "[MASK]"#単語を[MASK]に置き換える
print(words)

ベースボールがMASKに置き換わる。
次に単語を対応するインデックスに置き換える

In [ ]:
word_ids = tokenizer.convert_tokens_to_ids(words)#単語をインデックスに変換
word_tensor = torch.tensor([word_ids])#テンソルに変換
print(word_tensor)

それぞれ単語を表すIDに変換ができたのでBERTを使って予測をする

In [ ]:
msk_model = BertForMaskedLM.from_pretrained("bert-base-uncased")
msk_model.eval()#学習は行わないのでevalモードにする

x = word_tensor
y = msk_model(x)#予測
result = y[0]
print(result.size())#結果の形状

_, max_ids = torch.topk(result[0][msk_idx],k=5)#最も大きい5つの値
result_words = tokenizer.convert_ids_to_tokens(max_ids.tolist())#インデックスを単語に変換
print(result_words)




GPU対応はできないのでなし。対応する場合はx = tensor.cuda()を記述する
コードの流れは
①word_tensor→xに入れる→msk_modelに投入→resultで取り出す→予測結果を上位５つ表示
yのとこれで順伝番が行われてる
_,は本来、一番大きな値が入るが今回は使わないのでアンダースコア
②max_idsを(max_ids.tolist())でリストに変換→tokenizer.convert_ids_to_tokens()で単語に変換
③print(result_words)で表示
torch.sizeの中身解説
[1]はバッチサイズ
[11]は文章の中の単語数
[30522]は単語数
result_wordsの中身解説
[MASK]した部分がbaseballなので本来なら、resultの1番目はbaseballになるはずだが
出力は['football', 'basketball', 'golf', 'guitar', 'baseball']となった。
5番目に入っているだけでもすごいことだが少し偏りを感じる。
考察としてはモデル自体が海外より。海外の放課後の遊びは'football', 'basketball'が多いのかもしれない
[MASK]する部分を変えると100%の精度が出るかもしれない